#### This juypter notebook shows the process of making the pandas table for streamlit. This is **not** the latest version of the code.

## imports :  ]

In [16]:
import json

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import re

## Prizepicks Table

This is cleaning up the JSON file that contains all the data on PrizePicks. The goal is to make a Pandas dataframe from this data.

In [17]:
with open("prizepicks_json.json") as f:
    jsonobj = json.load(f)

count = 0
data_attributes = []

for item in jsonobj['data']:
    example = jsonobj['data'][count]
    data_attributes.append(example)
    count += 1

df_attr = pd.json_normalize(data_attributes)
df_attr = df_attr.rename(columns={'relationships.new_player.data.id': 'Player Name'})

count2 = 0
data_included = []

for item in jsonobj['included']:
    example2 = jsonobj['included'][count2]
    data_included.append(example2)
    count2 += 1

df_included = pd.json_normalize(data_included)

df_included = df_included.rename(columns={'id': 'Player Name'})

inner_join = pd.merge(df_attr, df_included, on='Player Name', how='inner')

Formatting the Prizepicks table

In [18]:
PRIZEPICKS_table = inner_join[['attributes.name', 'attributes.team', 'attributes.league', 'attributes.stat_type', 'attributes.line_score', 'attributes.updated_at']]
PRIZEPICKS_table.columns = ['Player', 'Team', 'Sport', 'Stat Type', 'Line', 'Last Updated']
PRIZEPICKS_table['Last Updated'] = PRIZEPICKS_table['Last Updated'].str.extract(r'(.*?)-\d+:')
PRIZEPICKS_table['Last Updated'] = pd.to_datetime(PRIZEPICKS_table['Last Updated'])
PRIZEPICKS_table['Last Updated'] = PRIZEPICKS_table['Last Updated'].dt.strftime('%m/%d/%y %I:%M %p')

This table shows all the current plays on Prizepicks

In [19]:
PRIZEPICKS_table

,Player,Team,Sport,Stat Type,Line,Last Updated
0,Terry Collins,MVSU,CBB,Rebounds,4.0,11/05/22 11:28 PM
1,Terry Collins,MVSU,CBB,Points,12.5,11/05/22 08:23 PM
2,Terry Collins,MVSU,CBB,Pts+Rebs+Asts,18.5,11/06/22 01:11 AM
3,Pass Attempts,NaN,NaN,Fantasy Score,76.5,11/05/22 10:51 PM
4,Chase Elliott,Chase Elliott,NASCAR,Fantasy Score,76.5,11/05/22 10:51 PM
...,...,...,...,...,...,...
1407,Gustav Forsling,FLA,NHL,Shots On Goal,3.0,11/06/22 03:04 AM
1408,Troy Terry,ANA,NHL,Shots On Goal,3.0,11/06/22 03:20 AM
1409,Carter Verhaeghe,FLA,NHL,Shots On Goal,2.5,11/06/22 02:55 AM
1410,Anton Lundell,FLA,NHL,Shots On Goal,2.5,11/06/22 04:02 AM


## NBA

Clean the NBA data from Draftkings by using regular expressions.

In [20]:
save_path = 'Draftkings_data\\'
# NBA Points
with open(save_path+'draftkings_NBA_PTS.txt', 'r') as file:
    NBA_PTS = file.read()
NBA_PTS = NBA_PTS.replace("âˆ’", "-")

NBA_PTS_Line = re.findall('line">(\d+.\d+)', str(NBA_PTS))

NBA_PTS_Odds = re.findall('color">(.\d+)', str(NBA_PTS))

NBA_PTS_Players = re.findall('row-name">(.*?)</', str(NBA_PTS))

# NBA Rebounds
with open(save_path+'draftkings_NBA_Rebounds.txt', 'r') as file:
    Rebounds = file.read()
Rebounds = Rebounds.replace("âˆ’", "-")

Rebounds_Line = re.findall('line">(\d+.\d+)', str(Rebounds))

Rebounds_Odds = re.findall('color">(.\d+)', str(Rebounds))

Rebounds_Players = re.findall('row-name">(.*?)</', str(Rebounds))

# Example 
print(NBA_PTS_Players)

['Anthony Davis', 'LeBron James', 'Lonnie Walker IV', 'Patrick Beverley', 'Troy Brown Jr.', 'Desmond Bane', 'Dillon Brooks', 'Ja Morant', 'Kristaps Porzingis', 'Kyle Kuzma', 'Santi Aldama', 'Steven Adams', 'Ivica Zubac', 'Jarred Vanderbilt', 'Jordan Clarkson', 'Kelly Olynyk', 'Lauri Markkanen', 'Marcus Morris Sr.', 'Mike Conley', 'Paul George', 'Reggie Jackson']


Our goal is to turn turn this data from Draftkings into a Pandas dataframe. First we need to seperate the Odds from the *Under* and *Over*. We can do this by seperating on every *2nd* number.

In [21]:
print(NBA_PTS_Odds)

['-110', '-115', '-125', '-105', '-120', '-105', '-140', '+105', '-125', '-105', '-110', '-120', '-105', '-125', '-115', '-115', '-105', '-125', '-105', '-125', '+100', '-125', '-115', '-110', '-120', '-110', '+100', '-130', '-105', '-125', '-125', '-105', '-115', '-110', '-105', '-120', '-130', '+100', '-115', '-115', '-125', '-105']


In [22]:
Upd_Odds13 = []
Upd_Lines13 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in NBA_PTS_Line:
    Upd_Lines13.append(NBA_PTS_Line[First:Second])
    First += 2
    Second += 2

for i in NBA_PTS_Odds:
    Upd_Odds13.append(NBA_PTS_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


NBA_PTS_Odds_DF = pd.DataFrame(Upd_Odds13)
NBA_PTS_Odds_DF = NBA_PTS_Odds_DF.dropna()

NBA_PTS_Lines_DF = pd.DataFrame(Upd_Lines13)
NBA_PTS_Lines_DF = NBA_PTS_Lines_DF.dropna()
NBA_PTS_Lines_DF = NBA_PTS_Lines_DF.drop([0],axis=1)

# UNDER and OVER odds in their own columns
print(NBA_PTS_Odds_DF)

       0     1
0   -110  -115
1   -125  -105
2   -120  -105
3   -140  +105
4   -125  -105
5   -110  -120
6   -105  -125
7   -115  -115
8   -105  -125
9   -105  -125
10  +100  -125
11  -115  -110
12  -120  -110
13  +100  -130
14  -105  -125
15  -125  -105
16  -115  -110
17  -105  -120
18  -130  +100
19  -115  -115
20  -125  -105


In [23]:
Upd_Odds14 = []
Upd_Lines14 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in Rebounds_Line:
    Upd_Lines14.append(Rebounds_Line[First:Second])
    First += 2
    Second += 2

for i in Rebounds_Odds:
    Upd_Odds14.append(Rebounds_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


Rebounds_Odds_DF = pd.DataFrame(Upd_Odds14)
Rebounds_Odds_DF = Rebounds_Odds_DF.dropna()

Rebounds_Lines_DF = pd.DataFrame(Upd_Lines14)
Rebounds_Lines_DF = Rebounds_Lines_DF.dropna()
Rebounds_Lines_DF = Rebounds_Lines_DF.drop([0],axis=1)

Now, we can create the dataframes for the Draftkings data. We are creating them by joining the Players, Lines, and Odds dataframes.

In [24]:
NBA_PTS_DK = pd.DataFrame(list(zip(NBA_PTS_Players)),
               columns =['Name'])

NBA_PTS_DK = NBA_PTS_DK.join(NBA_PTS_Lines_DF)
NBA_PTS_DK = NBA_PTS_DK.join(NBA_PTS_Odds_DF, lsuffix="fa", rsuffix="werf")

NBA_PTS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

NBA_PTS_DK[['O Odds', 'Line', 'U Odds']] = NBA_PTS_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)
print(NBA_PTS_DK)

                Player  Line  O Odds  U Odds
0        Anthony Davis  20.5    -110    -115
1         LeBron James  26.5    -125    -105
2     Lonnie Walker IV  15.5    -120    -105
3     Patrick Beverley   5.5    -140     105
4       Troy Brown Jr.   8.5    -125    -105
5         Desmond Bane  20.5    -110    -120
6        Dillon Brooks  17.5    -105    -125
7            Ja Morant  27.5    -115    -115
8   Kristaps Porzingis  23.5    -105    -125
9           Kyle Kuzma  18.5    -105    -125
10        Santi Aldama  10.5     100    -125
11        Steven Adams   6.5    -115    -110
12         Ivica Zubac  11.5    -120    -110
13   Jarred Vanderbilt   8.5     100    -130
14     Jordan Clarkson  18.5    -105    -125
15        Kelly Olynyk  11.5    -125    -105
16     Lauri Markkanen  19.5    -115    -110
17   Marcus Morris Sr.  14.5    -105    -120
18         Mike Conley  11.5    -130     100
19         Paul George  24.5    -115    -115
20      Reggie Jackson  13.5    -125    -105


In [25]:
Rebounds_DK = pd.DataFrame(list(zip(Rebounds_Players)),
               columns =['Name'])

Rebounds_DK = Rebounds_DK.join(Rebounds_Lines_DF)
Rebounds_DK = Rebounds_DK.join(Rebounds_Odds_DF, lsuffix="fa", rsuffix="werf")

Rebounds_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

Rebounds_DK[['O Odds', 'Line', 'U Odds']] = Rebounds_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

Now I am formatting the dataframes by adding a `Probability` column with a formula. And the `Favor` column will decide which side has the higher probability (Over/Under).

In [26]:
NBA_PTS_DK['O Prob'] = (NBA_PTS_DK['O Odds'].abs()/(NBA_PTS_DK['O Odds'].abs()+100)*100).round(1).where(NBA_PTS_DK['O Odds'] < 0, 100/(NBA_PTS_DK['O Odds']+100)*100).round(1)
NBA_PTS_DK['U Prob'] = (NBA_PTS_DK['U Odds'].abs()/(NBA_PTS_DK['U Odds'].abs()+100)*100).round(1).where(NBA_PTS_DK['U Odds'] < 0, 100/(NBA_PTS_DK['U Odds']+100)*100).round(1)
NBA_PTS_DK['Favor'] = np.where(NBA_PTS_DK['O Prob']> NBA_PTS_DK['U Prob'], "Over", "Under")

NBA_PTS_DK.head()

,Player,Line,O Odds,U Odds,O Prob,U Prob,Favor
0,Anthony Davis,20.5,-110,-115,52.4,53.5,Under
1,LeBron James,26.5,-125,-105,55.6,51.2,Over
2,Lonnie Walker IV,15.5,-120,-105,54.5,51.2,Over
3,Patrick Beverley,5.5,-140,105,58.3,48.8,Over
4,Troy Brown Jr.,8.5,-125,-105,55.6,51.2,Over


In [27]:
Rebounds_DK['O Prob'] = (Rebounds_DK['O Odds'].abs()/(Rebounds_DK['O Odds'].abs()+100)*100).round(1).where(Rebounds_DK['O Odds'] < 0, 100/(Rebounds_DK['O Odds']+100)*100).round(1)
Rebounds_DK['U Prob'] = (Rebounds_DK['U Odds'].abs()/(Rebounds_DK['U Odds'].abs()+100)*100).round(1).where(Rebounds_DK['U Odds'] < 0, 100/(Rebounds_DK['U Odds']+100)*100).round(1)
Rebounds_DK['Favor'] = np.where(Rebounds_DK['O Prob']> Rebounds_DK['U Prob'], "Over", "Under")

Rebounds_DK.head()

,Player,Line,O Odds,U Odds,O Prob,U Prob,Favor
0,Anthony Davis,9.5,-110,-120,52.4,54.5,Under
1,LeBron James,7.5,-105,-125,51.2,55.6,Under
2,Lonnie Walker IV,2.5,-145,110,59.2,47.6,Over
3,Patrick Beverley,3.5,-105,-125,51.2,55.6,Under
4,Troy Brown Jr.,4.5,-120,-110,54.5,52.4,Over


Lastly, we have to match similar data from Draftkings and Prizepicks. We need to make sure we are *Fuzzy matching* on the same stat type and sport to prevent incorrect matches.

In [28]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

NBA_PTS_TABLE = fuzzy_merge(PRIZEPICKS_table, NBA_PTS_DK, 'Player', 'Player', threshold=95)

NBA_PTS_TABLE = NBA_PTS_TABLE[NBA_PTS_TABLE['Stat Type'].str.contains('Points')]
NBA_PTS_TABLE = NBA_PTS_TABLE[NBA_PTS_TABLE['Sport'].str.endswith('NBA', na=False)]
NBA_PTS_TABLE['Player'] = NBA_PTS_TABLE['matches']

NBA_PTS_TABLE = NBA_PTS_TABLE.merge(NBA_PTS_DK, on='Player')

NBA_PTS_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Odds,U Odds,O Prob,U Prob,Favor
0,LeBron James,LAL,NBA,Points,26.5,11/06/22 10:47 AM,LeBron James,26.5,-125,-105,55.6,51.2,Over
1,Anthony Davis,LAL,NBA,Points,20.5,11/06/22 10:46 AM,Anthony Davis,20.5,-110,-115,52.4,53.5,Under
2,Lonnie Walker IV,LAL,NBA,Points,15.5,11/06/22 10:47 AM,Lonnie Walker IV,15.5,-120,-105,54.5,51.2,Over
3,Patrick Beverley,LAL,NBA,Points,5.5,11/06/22 10:46 AM,Patrick Beverley,5.5,-140,105,58.3,48.8,Over
4,Ja Morant,MEM,NBA,Points,27.5,11/06/22 10:37 AM,Ja Morant,27.5,-115,-115,53.5,53.5,Under


In [29]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

Rebounds_TABLE = fuzzy_merge(PRIZEPICKS_table, Rebounds_DK, 'Player', 'Player', threshold=95)

Rebounds_TABLE = Rebounds_TABLE[Rebounds_TABLE['Stat Type'].str.contains('Rebounds')]
Rebounds_TABLE = Rebounds_TABLE[Rebounds_TABLE['Sport'].str.endswith('NBA')]
Rebounds_TABLE['Player'] = Rebounds_TABLE['matches']

Rebounds_TABLE = Rebounds_TABLE.merge(Rebounds_DK, on='Player')

Rebounds_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Odds,U Odds,O Prob,U Prob,Favor
0,LeBron James,LAL,NBA,Rebounds,7.5,11/06/22 10:49 AM,LeBron James,7.5,-105,-125,51.2,55.6,Under
1,Anthony Davis,LAL,NBA,Rebounds,9.5,11/06/22 10:50 AM,Anthony Davis,9.5,-110,-120,52.4,54.5,Under
2,Patrick Beverley,LAL,NBA,Rebounds,3.5,11/06/22 10:50 AM,Patrick Beverley,3.5,-105,-125,51.2,55.6,Under
3,Santi Aldama,MEM,NBA,Rebounds,5.5,11/06/22 10:54 AM,Santi Aldama,5.5,-115,-115,53.5,53.5,Under


## Main table

Joining all the dataframes together as well as final formatting of the **main** table. 

In [30]:
MAIN_TABLE = pd.concat([
    NBA_PTS_TABLE, Rebounds_TABLE])

# Checking if the line is better on Prizepicks, if it is then apply a '+' sign
MAIN_TABLE['Favor'] = np.where((MAIN_TABLE['Line_y'] > MAIN_TABLE['Line_x']) & (MAIN_TABLE['Favor'] == 'Over'), MAIN_TABLE['Favor']+" +", MAIN_TABLE['Favor'])
MAIN_TABLE['Favor'] = np.where((MAIN_TABLE['Line_y'] < MAIN_TABLE['Line_x']) & (MAIN_TABLE['Favor'] == 'Under'), MAIN_TABLE['Favor']+" +", MAIN_TABLE['Favor'])  

MAIN_TABLE['Line_y'] = np.where((MAIN_TABLE['Line_x']!= MAIN_TABLE['Line_y']) & (MAIN_TABLE['Favor'].str.contains(" +") == False), pd.NaT, "Under")
MAIN_TABLE['Player'].replace('', np.nan, inplace=True)
MAIN_TABLE = MAIN_TABLE.dropna()
MAIN_TABLE['Probability'] = np.where(MAIN_TABLE['O Prob']== MAIN_TABLE['U Prob'], 50, MAIN_TABLE[["O Prob", "U Prob"]].max(axis=1))
MAIN_TABLE = MAIN_TABLE.drop(columns=['matches', 'Line_y', 'O Prob', 'U Prob'], axis=1)
MAIN_TABLE = MAIN_TABLE.rename(columns={'Line_x': 'Line', 'O Odds': 'Over', 'U Odds': 'Under'})
MAIN_TABLE = MAIN_TABLE[['Player', 'Team', 'Sport', 'Stat Type', 'Last Updated', 'Line','Over', 'Under', 'Probability', 'Favor']]
MAIN_TABLE.to_csv("MAIN_TABLE.csv", index=False)
MAIN_TABLE


,Player,Team,Sport,Stat Type,Last Updated,Line,Over,Under,Probability,Favor
0,LeBron James,LAL,NBA,Points,11/06/22 10:47 AM,26.5,-125,-105,55.6,Over
1,Anthony Davis,LAL,NBA,Points,11/06/22 10:46 AM,20.5,-110,-115,53.5,Under
2,Lonnie Walker IV,LAL,NBA,Points,11/06/22 10:47 AM,15.5,-120,-105,54.5,Over
3,Patrick Beverley,LAL,NBA,Points,11/06/22 10:46 AM,5.5,-140,105,58.3,Over
4,Ja Morant,MEM,NBA,Points,11/06/22 10:37 AM,27.5,-115,-115,50.0,Under
5,Desmond Bane,MEM,NBA,Points,11/06/22 10:37 AM,20.5,-110,-120,54.5,Under
6,Steven Adams,MEM,NBA,Points,11/06/22 10:37 AM,6.5,-115,-110,53.5,Over
7,Dillon Brooks,MEM,NBA,Points,11/06/22 10:38 AM,17.5,-105,-125,55.6,Under
8,Santi Aldama,MEM,NBA,Points,11/06/22 10:56 AM,10.5,100,-125,55.6,Under
0,LeBron James,LAL,NBA,Rebounds,11/06/22 10:49 AM,7.5,-105,-125,55.6,Under
